In [14]:
from bs4 import BeautifulSoup as bs
import requests
import os
from splinter import Browser
from bs4 import BeautifulSoup
import cssutils
import pandas as pd
from tabulate import tabulate
import json
#from bson.json_util import loads, dumps

# NASA Mars News

In [15]:
#Link to page then pull the request then use beautiful soup to scrape it
page_link  = "https://mars.nasa.gov/news/8426/nasa-garners-7-webby-award-nominations/"
page_response = requests.get(page_link, timeout=5)
nasa_page_content = bs(page_response.content, "html.parser")

In [16]:
nasa_title= nasa_page_content.title.text
nasa_title

'NASA Garners 7 Webby Award Nominations – NASA’s Mars Exploration Program '

In [17]:
nasa_page_content.prettify()

'<!DOCTYPE html>\n<html lang="en" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml">\n <head>\n  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>\n  <!-- Always force latest IE rendering engine or request Chrome Frame -->\n  <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>\n  <script type="text/javascript">\n   window.NREUM||(NREUM={});NREUM.info={"beacon":"bam.nr-data.net","errorBeacon":"bam.nr-data.net","licenseKey":"5e33925808","applicationID":"59562082","transactionName":"JVcPR0MLWApSRU1eAQVVEhxfAUMVaF4WVg4VFxJbXhM=","queueTime":0,"applicationTime":735,"agent":""}\n  </script>\n  <script type="text/javascript">\n   (window.NREUM||(NREUM={})).loader_config={xpid:"VQcPUlZTDxAFXVRUBQEPVA=="};window.NREUM||(NREUM={}),__nr_require=function(t,n,e){function r(e){if(!n[e]){var o=n[e]={exports:{}};t[e][0].call(o.exports,function(n){var o=t[e][1][n];return r(o||n)},o,o.exports)}return n[e].exports}if("function"==typeof __nr_require)return __nr_require;for(

In [18]:
#find all paragraphs
nasa_page_content.find_all("p")

[<p>Seven NASA websites and social media efforts are nominated for the 2019 Webby People's Voice Awards. Public voting continues through April 18. Credit: NASA/JPL-Caltech</p>,
 <p>Want to talk about how to land spacecraft onto other planets? We're here for you. Looking for the latest news or in-depth stories about science, technology and exploration? We've got those, too.</p>,
 <p>And apparently, people have noticed. Seven of NASA's digital properties have been nominated for the 2019 Webby Awards, which recognize excellence in digital communications. Three others have been <span>designated</span> honorees in this year's awards.</p>,
 <p>"We're really excited to have all of these efforts recognized," said Bettina Inclán, NASA's associate administrator for communications. "They represent hundreds of people across NASA who are committed to bringing the excitement of exploration to everyone."</p>,
 <p>The social media efforts for NASA's <a href="https://twitter.com/NASAInSight" target="_b

In [19]:
#since there are multiple paragraphs I will loop through them all and store to a list
nasa_textContent = []
for i in range(0, 14):
    nasa_paragraphs = nasa_page_content.find_all("p")[i].text
    nasa_textContent.append(nasa_paragraphs)

In [20]:
#first paragraph
nasa_textContent[0]

"Seven NASA websites and social media efforts are nominated for the 2019 Webby People's Voice Awards. Public voting continues through April 18. Credit: NASA/JPL-Caltech"

In [21]:
#last paragraph
nasa_textContent[13]

'Nominees include four JPL projects: the solar system and climate websites, InSight social media, and a 360-degree Earth video. Public voting closes April 18, 2019.'

# JPL Mars Space Images - Featured Image

In [22]:
executable_path = {'executable_path': 'chromedriver.exe'}
mars_browser = Browser('chrome', **executable_path, headless=False)

In [23]:
#use browser to visist url
mars_url="https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
mars_browser.visit(mars_url)

In [24]:
#Use BS to parse data
mars_html = mars_browser.html
mars_soup = BeautifulSoup(mars_html, 'html.parser')

In [25]:
#this will find the src for the background image but this isnt a url
div_style = mars_soup.find('article')['style']
style = cssutils.parseStyle(div_style)
image_url = style['background-image']
image_url = image_url.replace('url(', '').replace(')', '')

#tag the ending to the base url
featured_image_url = f'www.jpl.nasa.gov{image_url}'
print(featured_image_url)
mars_browser.quit()

www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA11777-1920x1200.jpg


# Mars Weather

In [26]:
twitter_browser = Browser('chrome', **executable_path, headless=False)
twitter_url = "https://twitter.com/marswxreport?lang=en"
twitter_browser.visit(twitter_url)
twitter_html = twitter_browser.html
twitter_soup = BeautifulSoup(twitter_html, 'html.parser')


In [27]:
#this will find the tweets about weather
mars_weather=[]
weather_info_list = []
for weather_info in twitter_soup.find_all('p',class_="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text"):
        weather_info_list.append(weather_info.text.strip())

# Find the latest tweet about weather
mars_weather = weather_info_list[0]

#Clean up the data a little to remove excess data at end
mars_weather = mars_weather[:-30]
print(mars_weather)
twitter_browser.quit()

InSight sol 137 (2019-04-16) low -97.0ºC (-142.7ºF) high -15.9ºC (3.4ºF)
winds from the SW at 4.3 m/s (9.7 mph) gusting to 12.4 m/s (27.7 mph)
pressure at 7.30


# Mars Facts

In [28]:
facts_url = "https://space-facts.com/mars/"

In [29]:
#USE BS to scrape the table data from website and print as table string
res = requests.get(facts_url)
soup = BeautifulSoup(res.content,'lxml')
facts_table = soup.find_all('table')[0] 
facts_df = pd.read_html(str(facts_table))
print(facts_df)


[                      0                              1
0  Equatorial Diameter:                       6,792 km
1       Polar Diameter:                       6,752 km
2                 Mass:  6.42 x 10^23 kg (10.7% Earth)
3                Moons:            2 (Phobos & Deimos)
4       Orbit Distance:       227,943,824 km (1.52 AU)
5         Orbit Period:           687 days (1.9 years)
6  Surface Temperature:                  -153 to 20 °C
7         First Record:              2nd millennium BC
8          Recorded By:           Egyptian astronomers]


In [30]:
#convert tp JSON
print(facts_df[0].to_json(orient='records'))

[{"0":"Equatorial Diameter:","1":"6,792 km"},{"0":"Polar Diameter:","1":"6,752 km"},{"0":"Mass:","1":"6.42 x 10^23 kg (10.7% Earth)"},{"0":"Moons:","1":"2 (Phobos & Deimos)"},{"0":"Orbit Distance:","1":"227,943,824 km (1.52 AU)"},{"0":"Orbit Period:","1":"687 days (1.9 years)"},{"0":"Surface Temperature:","1":"-153 to 20 \u00b0C"},{"0":"First Record:","1":"2nd millennium BC"},{"0":"Recorded By:","1":"Egyptian astronomers"}]


In [31]:
#print cleaner looking table
print( tabulate(facts_df[0], headers='keys') )

    0                     1
--  --------------------  -----------------------------
 0  Equatorial Diameter:  6,792 km
 1  Polar Diameter:       6,752 km
 2  Mass:                 6.42 x 10^23 kg (10.7% Earth)
 3  Moons:                2 (Phobos & Deimos)
 4  Orbit Distance:       227,943,824 km (1.52 AU)
 5  Orbit Period:         687 days (1.9 years)
 6  Surface Temperature:  -153 to 20 °C
 7  First Record:         2nd millennium BC
 8  Recorded By:          Egyptian astronomers


# Mars Hemispheres

In [32]:
#set main url
hem_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"

In [33]:
#use browser to visist url
hem_browser = Browser('chrome', **executable_path, headless=False)
hem_browser.visit(hem_url)

hem_html = hem_browser.html
hem_soup = bs(hem_html, 'html.parser')
    
# set open list to append images to
hemisphere_image_urls = []

products = hem_soup.find('div', class_='result-list')
hemispheres = products.find_all('div', class_='item')
    
#find the links to hemispheres, click, then browse those for imge url and append to open list
for hemisphere in hemispheres:
    title = hemisphere.find('div', class_='description')

    title_text = title.a.text
    title_text = title_text.replace(' Enhanced', '')
    hem_browser.click_link_by_partial_text(title_text)

    hem_html = hem_browser.html
    hem_soup = bs(hem_html, 'html.parser')

    image = hem_soup.find('div', class_='downloads').find('ul').find('li')
    img_url = image.a['href']

    hemisphere_image_urls.append({'title': title_text, 'img_url': img_url})

    hem_browser.click_link_by_partial_text('Back')
hem_browser.quit()

In [34]:
hemisphere_image_urls


[{'title': 'Cerberus Hemisphere',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]

In [42]:
from flask import Flask
from bs4 import BeautifulSoup as bs
import requests
import os
from splinter import Browser
from bs4 import BeautifulSoup
import cssutils
import pandas as pd
from tabulate import tabulate
import json
#from bson import json_util


executable_path = {'executable_path': 'chromedriver.exe'}
def marsNews():

    page_link  = "https://mars.nasa.gov/news/8426/nasa-garners-7-webby-award-nominations/"
    page_response = requests.get(page_link, timeout=5)
    nasa_page_content = bs(page_response.content, "html.parser")

    nasa_title= nasa_page_content.title.text
    nasa_page_content.find_all("p")   
    nasa_textContent = []
    for i in range(0, 14):
        nasa_paragraphs = nasa_page_content.find_all("p")[i].text
        nasa_textContent.append(nasa_paragraphs)
    
    nasa_latest_news = nasa_textContent[0]
    news = f'Title:{nasa_title}, Latest News: {nasa_latest_news}'
#     news_json=json.loads(news)
    
    
   
    return news


def marsFeaturedImageURL():

    
    mars_browser = Browser('chrome', **executable_path, headless=False)

    mars_url="https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
    mars_browser.visit(mars_url)
    mars_html = mars_browser.html
    mars_soup = BeautifulSoup(mars_html, 'html.parser')
    div_style = mars_soup.find('article')['style']
    style = cssutils.parseStyle(div_style)
    image_url = style['background-image']
    image_url = image_url.replace('url(', '').replace(')', '')

    #tag the ending to the base url
    featured_image_url = f'www.jpl.nasa.gov{image_url}'
    
    #convert to json for mongo import
    image_json=json.dumps(featured_image_url)
    #return featured_image_url
    return image_json
    mars_browser.quit()

def marsWeather():
    twitter_browser = Browser('chrome', **executable_path, headless=False)
    twitter_url = "https://twitter.com/marswxreport?lang=en"
    twitter_browser.visit(twitter_url)
    twitter_html = twitter_browser.html
    twitter_soup = BeautifulSoup(twitter_html, 'html.parser')

    #this will find the tweets about weather
    mars_weather=[]
    weather_info_list = []
    for weather_info in twitter_soup.find_all('p',class_="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text"):
            weather_info_list.append(weather_info.text.strip())

    # Find the latest tweet about weather
    mars_weather = weather_info_list[0]

    #Clean up the data a little to remove excess data at end
    mars_weather = mars_weather[:-30]
    
    #convert to json for mongo import
    weather_json=json.dumps(mars_weather)
    #return featured_image_url
    return weather_json
    mars_browser.quit()
    #return mars_weather
    twitter_browser.quit()

def marsFacts():
    facts_url = "https://space-facts.com/mars/"
    #USE BS to scrape the table data from website and print as table string
#     res = requests.get(facts_url)
#     soup = BeautifulSoup(res.content,'lxml')
#     facts_table = soup.find_all('table') 
    facts = pd.read_html(facts_url)[0]
    facts.columns=["Description","Value"]
    facts.set_index("Description", inplace=True)
    
#     records = json.dumps(facts)
    #try to change column names and reset index
#     facts = facts.rename({0:"Description",1:"Value"}, axis="columns")
#     facts.set_index("Description", inplace=True)
#     facts.columns=['des','value']
#     facts.set_index('des', inplace=True)
    #convert to json for mongo import
    #facts_json = facts.to_json()
    return facts.to_html()
    

def marsHems():
    #set main url
    hem_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
    #use browser to visist url
    hem_browser = Browser('chrome', **executable_path, headless=False)
    hem_browser.visit(hem_url)

    hem_html = hem_browser.html
    hem_soup = bs(hem_html, 'html.parser')
        
    # set open list to append images to
    hemisphere_image_urls = []

    products = hem_soup.find('div', class_='result-list')
    hemispheres = products.find_all('div', class_='item')
        
    #find the links to hemispheres, click, then browse those for imge url and append to open list
    for hemisphere in hemispheres:
        title = hemisphere.find('div', class_='description')

        title_text = title.a.text
        title_text = title_text.replace(' Enhanced', '')
        hem_browser.click_link_by_partial_text(title_text)

        hem_html = hem_browser.html
        hem_soup = bs(hem_html, 'html.parser')

        image = hem_soup.find('div', class_='downloads').find('ul').find('li')
        img_url = image.a['href']

        hemisphere_image_urls.append({'title': title_text, 'img_url': img_url})

        hem_browser.click_link_by_partial_text('Back')
    
    #convert to json for mongo import
    hems_json = json.dumps(hemisphere_image_urls)
    return hems_json
    #return hemisphere_image_urls
    hem_browser.quit()

    
   

In [43]:
app = Flask(__name__)
@app.route("/")

def scrape():
    "Main scrape functiona that will execute all other functions" 
    "and return a dictionary"

    mars_info = {}

    mars_info["news"] = marsNews()

    mars_info["featured_image_url"] = marsFeaturedImageURL()

    mars_info["weather"] = marsWeather()

    mars_info["facts"] = marsFacts()

    mars_info["mars_hemispheres"] = marsHems()

    
    mars_info={
        "news":marsNews(),
        "featured_image_url":marsFeaturedImageURL(),
        "weather":marsWeather(),
        "facts":marsFacts(),
        "mars_hemispheres":marsHems()
        }
    
    # return mars_data dict
    return mars_info

if __name__ == "__main__":  # run app
    app.debug = True


In [44]:
import pymongo
#Establish connection
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [46]:
mars_info={
        "news":marsNews(),
        "featured_image_url":marsFeaturedImageURL(),
        "weather":marsWeather(),
        "facts":marsFacts(),
        "mars_hemispheres":marsHems()
    
        }

TypeError: eval() arg 1 must be a string, bytes or code object

In [ ]:
#Create schema
marsDB = client["marsDB"]
#Drop artist collection
marsDB.scraped_data.drop()
#Create collection
mars_col = marsDB["scraped_data"]

In [ ]:
#Insert records into collection 
mars_col.insert(mars_info)